In [1]:
import pandas as pd
pd.options.display.max_columns = 100
import numpy as np

In [101]:
n_int = pd.read_csv('data/n_int.csv').set_index('PersonIdCode')

In [102]:
ppl = pd.read_csv('data/people_jonas.csv').set_index('PersonIdCode')

# Computing the median

Works on things like #words, #interventions etc.

In [103]:
def get_median(df, ppl, what):
    cols = df.columns.tolist()
    median_dict = {'what': what}
    for col in cols:
        sesh = df[col]
        sesh = strip_sesh(col, sesh, ppl)
        median = sesh.median()
        median_dict[col] = median
    return median_dict
        
def strip_sesh(col, sesh, ppl):
    #print(sesh.shape)
    for idcode, inter in sesh.iteritems():
        first_sesh = ppl.loc[idcode, 'FirstSession']
        if int(first_sesh) > int(col):
            sesh.pop(idcode)
    #print('-->'+str(sesh.shape))
    return sesh

In [104]:
d_int = get_median(n_int, ppl, 'median_interventions')

In [105]:
d_int

{'4601': 2.0,
 '4602': 3.0,
 '4603': 2.0,
 '4604': 2.0,
 '4605': 2.0,
 '4606': 1.0,
 '4607': 0.0,
 '4608': 2.0,
 '4609': 2.5,
 '4610': 0.0,
 '4611': 4.0,
 '4612': 4.0,
 '4613': 0.0,
 '4614': 3.5,
 '4615': 2.5,
 '4616': 2.0,
 '4617': 3.0,
 '4618': 0.0,
 '4619': 2.0,
 '4620': 3.0,
 '4701': 2.0,
 '4702': 3.0,
 '4703': 0.0,
 '4704': 3.0,
 '4705': 5.0,
 '4706': 3.0,
 '4707': 2.0,
 '4708': 3.0,
 '4709': 3.0,
 '4710': 4.0,
 '4711': 3.0,
 '4712': 1.0,
 '4713': 2.5,
 '4714': 3.0,
 '4715': 4.0,
 '4716': 5.0,
 '4717': 5.0,
 '4718': 6.0,
 '4801': 2.0,
 '4802': 4.0,
 '4803': 0.0,
 '4804': 4.0,
 '4805': 4.0,
 '4806': 2.5,
 '4807': 6.0,
 '4808': 0.0,
 '4809': 5.0,
 '4810': 0.0,
 '4811': 3.0,
 '4812': 4.0,
 '4813': 7.0,
 '4814': 5.0,
 '4815': 5.0,
 '4816': 4.0,
 '4817': 5.0,
 '4818': 1.0,
 '4819': 4.0,
 '4820': 5.0,
 '4901': 3.0,
 '4902': 4.0,
 '4903': 0.0,
 '4904': 5.0,
 '4905': 5.0,
 '4906': 4.0,
 '4907': 5.0,
 '4908': 0.5,
 '4909': 4.0,
 '4910': 5.0,
 '4911': 5.0,
 '4912': 6.0,
 '4913': 1.0,
 '4914

In [106]:
n_words = pd.read_csv('data/n_words_clean.csv').set_index('PersonIdCode')

In [107]:
d_words = get_median(n_words, ppl, 'median_words')

In [97]:
d_words.keys() == d_int.keys()

True

In [130]:
cols = n_words.columns.tolist()

In [131]:
cols.extend(['what'])

In [132]:
medians = pd.DataFrame(columns=cols)

In [138]:
medians = medians.append(d_int, ignore_index=True).append(d_words, ignore_index=True)

In [140]:
medians = medians.set_index('what')

In [142]:
medians.to_csv('data/medians.csv')

In [395]:
ppl.loc[2593].LastName

'de Buman'

In [397]:
ppl.loc[ppl.LastName == 'Carobbio Guscetti']

,PersonNumber,FirstName,LastName,GenderAsString,DateOfBirth,Age,AgeCategory,AgeCategoryText,NativeLanguage,CantonAbbreviation,CantonName,CouncilAbbreviation,DateElection,DateJoining,ParlGroupAbbreviation,ParlGroupName,ParlGroupFunctionText,PartyAbbreviation,PartyName,FirstSession,FirstSessionDate,Seniority,TotalWords,NbrSessions,TotalInterventions,InterventionPerSession
PersonIdCode,,,,,,,,,,,,,,,,,,,,,,,,,,
2666,3830,Marina,Carobbio Guscetti,f,12.06.1966,50,4,50-59,I,TI,Tessin,CN,2015-10-18T00:00:00,2015-11-30T00:00:00,S,Groupe socialiste,Membre,PSS,Parti socialiste suisse,4717,2007-06-04,3496,112343,47,265,5.638298


In [27]:
for i, j in s.iteritems():
    print(i,j)
    if i == 2287:
        print('bitch')
        s.pop(i)

2061 0
2129 2
2143 3
2147 9
2287 3
bitch
2292 2
2310 0
2323 2
2328 2
2370 2
2372 0
2385 6
2405 1
2417 0
2501 0
2510 0
2525 0
2530 0
2534 1
2536 0
2541 0
2546 0
2561 0
2565 0
2569 0
2573 0
2575 0
2580 0
2583 0
2584 0
2585 0
2588 0
2589 0
2593 0
2594 0
2597 0
2600 0
2601 0
2608 0
2613 0
2614 0
2616 0
2617 0
2618 0
2619 0
2621 0
2623 0
2627 0
2629 0
2630 0
2631 0
2632 0
2638 0
2641 0
2642 0
2646 0
2647 0
2648 0
2652 0
2654 0
2657 0
2660 0
2661 0
2664 0
2666 0
2667 0
2669 0
2670 0
2674 0
2675 0
2678 0
2679 0
2680 0
2681 0
2682 0
2683 0
2685 0
2686 0
2688 0
2689 0
2690 0
2691 0
2693 0
2694 0
2698 0
2700 0
2701 0
2702 0
2704 0
2705 0
2707 0
2708 0
2710 0
2713 0
2715 0
2717 0
2718 0
2719 0
2721 0
2723 0
2724 0
2726 0
2728 0
2735 0
2738 0
2740 0
2741 0
2743 0
2744 0
2745 0
2747 0
2748 0
2750 0
2751 0
2752 0
2754 0
2757 0
2758 0
2759 0
2760 0
2761 0
2762 0
2763 0
2764 0
2765 0
2766 0
2767 0
2768 0
2769 0
2770 0
2771 0
2772 0
2773 0
2774 0
2775 0
2776 0
2777 0
2778 0
2779 0
2780 0
2781 0
2783 0


In [28]:
s.shape

(251,)